## Closures and Decorators

```py
def func():
    ....
```
`def` is executed at runtime ie. functions are defined at runtime.

```py
def func():
    def local_func():
        print("hello world")
```
`local_func()` is defined everytime func() is called.

In [16]:
from pprint import pprint
store=[]
def sort_by_last_letter(strings):
    def last_letter(s):
        return s[-1]
    store.append(last_letter)
    return sorted(strings, key=last_letter)

In [20]:
sort_by_last_letter(["hello", "world", "from", "python3"])
sort_by_last_letter(["hello", "world", "from", "python3"])
sort_by_last_letter(["hello", "world", "from", "python3"])
sort_by_last_letter(["hello", "world", "from", "python3"])

['python3', 'world', 'from', 'hello']

In [22]:
# each call creates a new instance of `last_letter`
pprint(store)

[<function sort_by_last_letter.<locals>.last_letter at 0x11025cd08>,
 <function sort_by_last_letter.<locals>.last_letter at 0x11025cbf8>,
 <function sort_by_last_letter.<locals>.last_letter at 0x11025cd90>,
 <function sort_by_last_letter.<locals>.last_letter at 0x11025cc80>,
 <function sort_by_last_letter.<locals>.last_letter at 0x11025cae8>]


Local funcs follow the same rules as the normal funcs. They follow **LEGB rule** for name resolution.

Local funcs are not members of global funcs. ie  
`sort_by_last_letter.last_letter`   
does not work.

Local funcs helps in writing functional programming in python. As you can return functions from another functions.  
**First class functions** => powerful concept

### Closures

In [24]:
def adder(x):
    def add_x(y):
        return x+y
    return add_x

add3 = adder(3)
print(add3(5))

8


`add_x` remember the value of `x`.  
How ?  
When `add_x` is returned it retains the objects of the scope, which contains the variable `x`. This ability is called closures.  
The returned function *closes over* the objects of the scope.

If python retains the objects, where does it store the references of these objects ?  
In the attribute `__closure__`.

In [28]:
def enclosing():
    x = 'closed over'
    y = 'not closed over'
    def local_func():
        print(x)
    return local_func

lf = enclosing()
lf()

closed over


In [30]:
lf.__closure__  # stores the object x, y is not present and will be garbage collected

(<cell at 0x110263bb8: str object at 0x110291a70>,)

**How are closures useful ?**

One use case is *Function factory* ie. a function that generate new specialsed functions.  
For eg. `adder` function specified above can generate specialised functions like `add3`.

In [34]:
def raise_to(exp):
    def raise_to_exp(x):
        return pow(x, exp)
    return raise_to_exp

square = raise_to(2)
print(square.__closure__)
print(square(3))
print(square(4))

(<cell at 0x110297e88: int object at 0x10e1dcaf0>,)
9
16


### Decorators

Decorators enable us to modify or enhance functions without changing their definitions.

**Decorators are powerful and are used widely in Python.**  

In python, decorators are *implemented as callables which take and return other callables*.

Suppose you want to calculate the time taken by a function to execute.

In [36]:
import time


def timing_function(some_function):

    """
    Outputs the time a function takes
    to execute.
    """

    def wrapper(*args, **kwargs):   # accepts all positional args & keyword args
        t1 = time.time()
        some_function(*args, **kwargs)
        t2 = time.time()
        return "Time it took to run the function: " + str((t2 - t1)) + "\n"
    return wrapper

def sum_till_10000():
    num_list = []
    for num in (range(0, 10000)):
        num_list.append(num)
    print("\nSum of all the numbers: " + str((sum(num_list))))

In [38]:
timing_function(sum_till_10000)()


Sum of all the numbers: 49995000


'Time it took to run the function: 0.0018477439880371094\n'

This is not a clean and scalable way to implement the timing functionality. Python provides us with decorators to implement such wrapping functions.

In [39]:
@timing_function
def sum_till_10000():
    num_list = []
    for num in (range(0, 10000)):
        num_list.append(num)
    print("\nSum of all the numbers: " + str((sum(num_list))))

In [40]:
sum_till_10000()


Sum of all the numbers: 49995000


'Time it took to run the function: 0.0017459392547607422\n'

**How does Python implement it ?**

The decorator `timing_funtion` takes a callable and returns a callable.  
When the decorator is used, Python executes `timing_function` and returns `wrapper`. Its then binds the returned `wrapper` function to the name of the original function ie `sum_till_10000`.

Another example can be creating a decorator called `@login_required` to make a view method authenticate the user.  


But functions are not the only callables in Python. **Classes are also callables.**  

To make a class a decorator, it must have the following format -
```py
class ClassDec:
    def __init__(self, f):
        self.f = f
        ...
        
    def __call__(self, *args, **kwargs):
        ...
        self.f(*args, **kwargs)
```

In [42]:
# CallCount decorator to count the number of times a func is called.
class CallCount:
    def __init__(self, f):
        self.f = f
        self.count = 0
        
    def __call__(self, *args, **kwargs):
        self.count += 1
        return self.f(*args, **kwargs)  # execute the func f & return the result

@CallCount
def hello(name):
    print("Hello, {}".format(name))
    
hello('One')
hello('Two')
hello('Three')

print(hello.count)

Hello, One
Hello, Two
Hello, Three
3


In [44]:
print(type(hello))
print(hello.__class__)
print(dir(hello))

<class '__main__.CallCount'>
<class '__main__.CallCount'>
['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'count', 'f']


After using class decorator, `hello` now points to an instance to class `CallCount` and is callable.  
Since `hello` is an instance, `hello.count` works !

**Class instances are also callables, and hence can be used as decorators !**

In [50]:
class Trace:
    def __init__(self):
        self.enabled = True
        
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self.enabled:
                print("Calling {}".format(f))
            return f(*args, **kwargs)           
        return wrap
    
tracer = Trace()   # instance is callable due to implementation of __call__

@tracer     # Class instance is used as a decorator
def rotate_list(l):
    return l[1:] + [l[0]]

rotate_list([1,2,3])

Calling <function rotate_list at 0x11024ee18>


[2, 3, 1]

*Using functions, classes and class instances as decorator gives a lot of flexibility to a developer.*  
Use it wisely !

**Multiple decorators**

```py
@decorator1
@decorator2
@decorator3
def some_func():
    ...
```

By using decorators, function's important metadata such as `__name__` and`'__doc__` is lost.  
Using `functools.wraps()` prserves this metadata.

In [57]:
def hello():
    "Print a well known message."
    print("Hello world")
    
print(hello.__name__)
print(hello.__doc__)
help(hello)

hello
Print a well known message.
Help on function hello in module __main__:

hello()
    Print a well known message.



In [58]:
# using a decorator
def noop(f):
    def wrapper():
        return f()
    return wrapper

@noop
def hello2():
    "Print a well known message."
    print("Hello world")
    
# function metadata is lost
print(hello2.__name__)
print(hello2.__doc__)
help(hello2)

wrapper
None
Help on function wrapper in module __main__:

wrapper()



In [59]:
import functools

def noop(f):
    @functools.wraps(f)   # this returns a func which acts as a decorator
    def wrapper():
        return f()
    return wrapper

@noop
def hello3():
    "Print a well known message."
    print("Hello world")
    
# function metadata is preserved
print(hello3.__name__)
print(hello3.__doc__)
help(hello)

hello3
Print a well known message.
Help on function hello in module __main__:

hello()
    Print a well known message.

